In [1]:
"""
ucd_atlas_loop.py
create and send sql query looping over atlas3d galaxies to find CEs
based on code from Aaron Deich
"""

from numpy import *
import urllib, urllib2

# Formulas:
# modulus = 5 * log10(a_dist * 1e6) - 5
# M_r = r - modulus
# size = petroR50_r / 3600 / 180 * pi * a_dist * 1e3

def writer(name,host,data):
    # Function to write data to a file
    f = open(name, 'a')             #defines the file to be opened as appendable
    #f.write('\n' + host + '\n')     #formatting for hosts
    f.write(data)                   #writes in data
    f.close()                       #tells python to close file
    return writer

def create_SQL_query(RA, DEC, distance, radius):
    # creates SQL query to search a radius (in kpc) around an object
    query_str = 'SELECT p.objID,p.ra,p.dec '#,p.u,p.g,p.r,p.i,p.z,p.petroR90_r / p.petroR50_r as Concentration, p.expAB_r, p.deVAB_r '
    query_str = ''.join([query_str, 'FROM PhotoObj p JOIN dbo.fGetNearbyObjEq('])
    query_str = ''.join([query_str, str(RA), ',', str(DEC), ',', str(radius), '*60*180/3.15159/1000/', str(distance)])
    query_str = ''.join([query_str, ') AS GN ON p.objID = GN.objID WHERE p.g-p.i between 0.9 and 1.4 AND p.type = 3 AND '])
    query_str = ''.join([query_str, '(p.petroR50_r'])
    #query_str = ''.join([query_str, '(((p.deVRad_r * p.fracDeV_r + p.expRad_r * (1 - p.fracDeV_r)))'])
    query_str = ''.join([query_str, '/3600/180*3.14159*1000*'])
    query_str = ''.join([query_str, str(distance), ') between 0.04 and 1'])
    query_str = ''.join([query_str, ' AND p.r - (5*log10(', str(distance), '*1000000)) + 5 between -20 and -12'])
    query_str = ''.join([query_str, ' AND p.petroR90_r / p.petroR50_r between 2.15 AND 2.95'])
    #query_str = ''.join([query_str, ' AND p.expAB_r between 0.77 AND 1.0'])
    query_str = ''.join([query_str, ' AND p.deVAB_r between 0.81 AND 1.0'])
    query_str = ''.join([query_str, " AND (flags & dbo.fPhotoFlags('SATUR_CENTER')) = 0"])
    return query_str

def SDSS_request(SQL_query):
    # sends SQL query to DR7 SQL search, returns as csv
    sURL = 'http://cas.sdss.org/dr7/en/tools/search/x_sql.asp'
    values = {'cmd': SQL_query, 'format': 'csv'}
    data = urllib.urlencode(values)
    request = urllib2.Request(sURL, data)
    response = urllib2.urlopen(request)
    return response.read()

# gets locations of and distances to objects in Atlas3D database
host_obj=genfromtxt('cappellari_early_type.csv',delimiter=',',skip_header=1,dtype=None, names=['name','ra','dec','sbf','ned_d','virgo','v_hel','distance','M_K','A_B','T_type','log(re)'])

# search radius of 40kpc, with search_rad in arcmin
# radius = 40 * 1e3 * 60 * 180 / pi / distance
radius = 40 #kpc
print ''
n = 5 # IC1024

#--------------------------------------------------------------------
# Section for testing single host galaxies
#--------------------------------------------------------------------
# create sql query
#query_string = create_SQL_query(host_obj['ra'][n], host_obj['dec'][n], host_obj['distance'][n], radius)
# send SQL query to DR7 database
#data_string = SDSS_request(query_string)
#print name[n]
#print query_string, '\n\n', data_string

file_name = raw_input('Enter a name for your .csv file: ') + '.csv'
#file_name = 'ic1024_test'
op = open(file_name, 'w')
op.write('Complete list of found objects sorted by host galaxy; by Alex Colebaugh \n')
#op.write(data_string)
op.close()

query_string = ''
data_string = ''
full_data = ''

for obj in arange(0,len(host_obj['ra']),1):
#for obj in arange(0,10,1):
    query_string = create_SQL_query(host_obj['ra'][obj], host_obj['dec'][obj], host_obj['distance'][obj], radius) 
    #print sql_query_string
    data_string = SDSS_request(query_string)
    print host_obj['name'][obj]
    print data_string
    if data_string == 'No objects have been found':
        continue 
    else:
        #split_string = data_string.split('deVAB_r\n')[1]
        split_string = data_string.split('dec\n')[1]
        if split_string in full_data:
            continue
        else:
            full_data += split_string
writer(file_name,host_obj['name'][obj],full_data)

# ucds_concentration: fracDeV, concentration between 2.0 and 3.0, cuts 914 entries but also UCDs
# ucds_petro_conc: petro, concentration between 2.0 and 3.0 cuts 795 entries
# ucds_restricted: petro, concentration between 2.15 and 2.95 cuts 1249 entries
# ucds_expfit: petro, concentration between 2.15 and 2.95 cuts 2272 entries

IOError: cappellari_early_type.csv not found.